In [1]:
import pandas as pd
import random
import numpy as np
import glob
from collections import Counter
from collections import defaultdict

# This document generates sample sets to use as testable iterations for imputation accuracy

> we start with 75 animals availabe on all 3 chips

> next we get a subset of all angus animals  that doesn't overlap with this test set for each chip so we're not working with the full set every time (might change)

> then we'll make a list of each of these full angus sets) that fits the plink standard

> then we'll make a couple of random subsets of the 75 testers. 

In [3]:
! ls ../../GEN_DUMP/

227234.161117.12083.100_B.ID   58336.161117.7744.100_A.ID
227234.161117.12083.100_B.ped  58336.161117.7744.100_A.ped
227234.161117.1994.200_A.ID    777962.161117.1681.100_A.ID
227234.161117.1994.200_A.ped   777962.161117.1681.100_A.ped
58336.161117.1062.100_C.ID     777962.161117.219.200_B.ID
58336.161117.1062.100_C.ped    777962.161117.219.200_B.ped
58336.161117.127.100_B.ID      777962.161117.2779.200_A.ID
58336.161117.127.100_B.ped     777962.161117.2779.200_A.ped
58336.161117.1339.200_A.ID     777962.161117.417.100_B.ID
58336.161117.1339.200_A.ped    777962.161117.417.100_B.ped
58336.161117.34296.200_B.ID    DUMP_GEN_for_imputation.sql
58336.161117.34296.200_B.ped   process_gen_dump.txt


In [44]:
Angus = glob.glob('../hwe_filtered/*.100*.fam')
Angus

['../hwe_filtered/227234.161117.12083.100_B.fam',
 '../hwe_filtered/58336.161117.1062.100_C.fam',
 '../hwe_filtered/58336.161117.127.100_B.fam',
 '../hwe_filtered/58336.161117.7744.100_A.fam',
 '../hwe_filtered/777962.161117.1681.100_A.fam',
 '../hwe_filtered/777962.161117.417.100_B.fam']

In [48]:
def Breedmaker(blist):
    data = []
    for i in blist:
        array,date,count,brV,ftype = i.split('/')[-1].split('.')
        breed,version = brV.split('_')
        data.append([i,array,version,int(count),breed,date])
    return pd.DataFrame(data,columns=['fname','array','version','total','breed','date'])

In [49]:
angframe = Breedmaker(Angus)
perChip=angframe.groupby('array').sum()
print(perChip)

        total
array        
227234  12083
58336    8933
777962   2098


In [50]:
usenow = pd.Series({'227234':2000,'58336':2000,'777962':1000},name='usenow')

In [51]:
perChip = perChip.join(usenow)

In [52]:
angframe = angframe.join(perChip,rsuffix='sum',on='array')

In [53]:
angframe

,fname,array,version,total,breed,date,totalsum,usenow
0,../hwe_filtered/227234.161117.12083.100_B.fam,227234,B,12083,100,161117,12083,2000
1,../hwe_filtered/58336.161117.1062.100_C.fam,58336,C,1062,100,161117,8933,2000
2,../hwe_filtered/58336.161117.127.100_B.fam,58336,B,127,100,161117,8933,2000
3,../hwe_filtered/58336.161117.7744.100_A.fam,58336,A,7744,100,161117,8933,2000
4,../hwe_filtered/777962.161117.1681.100_A.fam,777962,A,1681,100,161117,2098,1000
5,../hwe_filtered/777962.161117.417.100_B.fam,777962,B,417,100,161117,2098,1000


In [55]:
angframe['percentofchip'] = angframe['total'] / angframe.totalsum
angframe['animal_use'] = angframe.percentofchip * angframe.usenow
angframe

,fname,array,version,total,breed,date,totalsum,usenow,percentofchip,animal_use
0,../hwe_filtered/227234.161117.12083.100_B.fam,227234,B,12083,100,161117,12083,2000,1.000000,2000.000000
1,../hwe_filtered/58336.161117.1062.100_C.fam,58336,C,1062,100,161117,8933,2000,0.118885,237.770066
2,../hwe_filtered/58336.161117.127.100_B.fam,58336,B,127,100,161117,8933,2000,0.014217,28.433897
3,../hwe_filtered/58336.161117.7744.100_A.fam,58336,A,7744,100,161117,8933,2000,0.866898,1733.796037
4,../hwe_filtered/777962.161117.1681.100_A.fam,777962,A,1681,100,161117,2098,1000,0.801239,801.239276
5,../hwe_filtered/777962.161117.417.100_B.fam,777962,B,417,100,161117,2098,1000,0.198761,198.760724


In [138]:
random.sample(range(10),5)

[2, 8, 5, 3, 6]

In [56]:
def idgrab(idfile):
    temp = []
    with open(idfile) as fp:
        for line in fp:
            name = line.split()[1]
            temp.append(name)
    return temp
#print(name)

In [57]:
allanimals = []
perarray = defaultdict(list)
for f in angframe.itertuples():
    print(f[1],f[-1],f[4])
    fname,touse,total,array = f[1],f[-1],f[4],f[2]
    fix = random.sample(range(total),int(touse))
    bix = idgrab(fname)
    perarray[array].append(bix)
    allanimals =allanimals + bix
    
fname,total,touse

../hwe_filtered/227234.161117.12083.100_B.fam 2000.0 12083
../hwe_filtered/58336.161117.1062.100_C.fam 237.770066047 1062
../hwe_filtered/58336.161117.127.100_B.fam 28.4338967872 127
../hwe_filtered/58336.161117.7744.100_A.fam 1733.79603717 7744
../hwe_filtered/777962.161117.1681.100_A.fam 801.2392755 1681
../hwe_filtered/777962.161117.417.100_B.fam 198.7607245 417


('../hwe_filtered/777962.161117.417.100_B.fam', 417, 198.76072449952338)

In [58]:
arraynames = {}
for i in perarray:
    arraynames[i] = [val for sublist in perarray[i] for val in sublist]
[len(i) for i in arraynames.values()]
#perarray['58336'][1][:20]

[11918, 1963, 8693]

In [59]:
arraynames = {}
for i in perarray:
    arraynames[i] = set([val for sublist in perarray[i] for val in sublist])
[len(i) for i in arraynames.values()]
animalrecords = {}

for i in arraynames:
    animalrecords[i] = defaultdict(int)
    for t in arraynames[i]:
        animalrecords[i][t]= animalrecords[i][t]+1

In [60]:
chipperanimal = pd.DataFrame(animalrecords)
chipperanimal.shape

(21065, 3)

In [61]:
chipperanimal.sum().sum()

22566.0

In [62]:
chipperanimal.fillna(0,inplace=True)
chipperanimal['sum'] = chipperanimal.sum(axis=1)
chipperanimal.head()

,227234,58336,777962,sum
AANUSAF000017102723,0.0,0.0,1.0,1.0
AANUSAF000017102725,0.0,0.0,1.0,1.0
AANUSAF000017102733,0.0,0.0,1.0,1.0
AANUSAF000017102750,0.0,0.0,1.0,1.0
AANUSAF000017102754,0.0,0.0,1.0,1.0


In [85]:
snp50set = chipperanimal[(chipperanimal['227234'] == 0) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 0)]
f250set = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 0) &(chipperanimal['777962'] == 0)]
hdset = chipperanimal[(chipperanimal['227234'] == 0) & (chipperanimal['58336'] == 0) &(chipperanimal['777962'] == 1)]
f250snp50 = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 0)]
f250HD = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 0) &(chipperanimal['777962'] == 1)]
HDsnp50 = chipperanimal[(chipperanimal['227234'] == 0) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 1)]
singlechip = chipperanimal[chipperanimal['sum'] == 1]
threechips = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 1)]

f250snp50.sample(20).index

Index(['UMCUSAU000000087678', 'AANUSAM000013543130', 'AANUSAM000013818774',
       'AANUSAM000013036690', 'AANUSAM000011216496', 'AANUSAM000012552610',
       'AANUSAM000010748955', 'AANUSAM000013664352', 'AANUSAM000010610838',
       'AANUSAM000002653264', 'AANUSAM000013817992', 'UMCUSAM000000080845',
       'UMCUSAU000000087721', 'AANUSAM000013195065', 'AANUSAM000010848959',
       'AANUSAM000010362904', 'AANUSAM000012915498', 'AANUSAM000012137203',
       'AANUSAM000015944142', 'AANUSAM000010086319'],
      dtype='object')

# This will be used for making random samples of animals who are not on multiple chips
# These animals should all be processed and then merged into one file, they can be pulled out of there. 
# duplicate animals can be merged into seperate file, or kept in their original files
# animals that are f250 and chip should perhaps be prioritized, but with only their f250 data, or with their full complement of data, but not for the animals being tested.  


In [94]:
print(snp50set.shape)
print(f250set.shape)
print(hdset.shape)
print (singlechip.shape)
print(f250snp50.shape)
print(HDsnp50.shape)
print(f250HD.shape)
print(threechips.shape)

(7347, 4)
(10554, 4)
(1730, 4)
(19631, 4)
(1201, 4)
(71, 4)
(95, 4)
(67, 4)


In [79]:
singlechip.head()

,227234,58336,777962,sum
AANUSAF000017102723,0.0,0.0,1.0,1.0
AANUSAF000017102725,0.0,0.0,1.0,1.0
AANUSAF000017102733,0.0,0.0,1.0,1.0
AANUSAF000017102750,0.0,0.0,1.0,1.0
AANUSAF000017102754,0.0,0.0,1.0,1.0


# Take all of the individual chip represented animals. 
# Take a subsample the animals on two chips, but take more of which type?
# don't include multiple chips on same animal for now but revist


In [119]:
#testf250snp50 = f250snp50.index.tolist() + threechips.index.tolist()
#random.shuffle(testf250snp50) #makes random order of it just once
#writef250_test = open('f250test.txt','w')
#for i in testf250snp50:
#    writef250_test.write(i + '\n')
#writef250_test.close()
len(testf250snp50)

1268

# Above cell writes out the test file in a randomized order. 
# Then we take the file and write out f20 and snp50 lists from it.
# each is with 300 animals only at snp50 and the rest from f250. 

In [120]:
! head f250test.txt

AANUSAM000008792721
AANUSAM000012574548
AANUSAM000010748976
AANUSAM000013699208
AANUSAM000009493757
AANUSAM000014213888
AANUSAM000014300213
AANUSAM000015784659
AANUSAM000010989617
AANUSAM000012309326


In [124]:
readf250_test = open('f250test.txt','r')
f250samp = [i.strip() for i in readf250_test.readlines()]
f250samp[:10]

['AANUSAM000008792721',
 'AANUSAM000012574548',
 'AANUSAM000010748976',
 'AANUSAM000013699208',
 'AANUSAM000009493757',
 'AANUSAM000014213888',
 'AANUSAM000014300213',
 'AANUSAM000015784659',
 'AANUSAM000010989617',
 'AANUSAM000012309326']

In [141]:
writef250_1_snp50 = open('f250_1_snp50.txt','w')
for i in f250samp[:300]:
    writef250_1_snp50.write(' '.join([i,i,'\n']))
writef250_1_snp50.close()
writef250_1_f250 = open('f250_1_f250.txt','w')
for i in f250samp[300:]:
    writef250_1_f250.write(' '.join([i,i,'\n']))
writef250_1_f250.close()

In [142]:
writef250_2_snp50 = open('f250_2_snp50.txt','w')
for i in f250samp[300:600]:
    writef250_2_snp50.write(' '.join([i,i,'\n']))
writef250_2_snp50.close()
writef250_2_f250 = open('f250_2_f250.txt','w')
setter = f250samp[:300] + f250samp[600:]
for i in setter:
    writef250_2_f250.write(' '.join([i,i,'\n']))
writef250_2_f250.close()

In [140]:
writef250_3_snp50 = open('f250_3_snp50.txt','w')
for i in f250samp[600:900]:
    writef250_3_snp50.write(' '.join([i,i,'\n']))
writef250_3_snp50.close()
writef250_3_f250 = open('f250_3_f250.txt','w')
setter = f250samp[:600] + f250samp[900:]
for i in setter:
    writef250_3_f250.write(' '.join([i,i,'\n']))
writef250_3_f250.close()

In [139]:
writef250_4_snp50 = open('f250_4_snp50.txt','w')
for i in f250samp[900:]:
    writef250_4_snp50.write(' '.join([i,i,'\n']))
writef250_4_snp50.close()
writef250_4_f250 = open('f250_4_f250.txt','w')
setter = f250samp[:900]
for i in setter:
    writef250_4_f250.write(' '.join([i,i,'\n']))
writef250_4_f250.close()

plink --merge-list hwe_filtered/allfilesmerged2.txt --keep ./dataprepper/Animals_Single_chip.txt --nonfounders --cow --make-bed --out merged_files/single_chip_100

In [90]:
singlechip_ids = singlechip.index.tolist()

writesingle = open('Animals_Single_chip.txt','w')
for i in singlechip_ids:
    writesingle.write(' '.join([i,i,'\n']))
writesingle.close()

plink --merge-list hwe_filtered/allfilesmerged2.txt --keep ./dataprepper/Animals_Single_chip.txt --nonfounders  --cow --make-bed --out merged_files/single_chip_100

In [91]:
! head Animals_Single_chip.txt

AANUSAF000017102723 AANUSAF000017102723 
AANUSAF000017102725 AANUSAF000017102725 
AANUSAF000017102733 AANUSAF000017102733 
AANUSAF000017102750 AANUSAF000017102750 
AANUSAF000017102754 AANUSAF000017102754 
AANUSAF000017102762 AANUSAF000017102762 
AANUSAF000017102763 AANUSAF000017102763 
AANUSAF000017102769 AANUSAF000017102769 
AANUSAF000017102772 AANUSAF000017102772 
AANUSAF000017152022 AANUSAF000017152022 


# see the snakefile f250 test for info on how this got used. 

# We need fewer single chip animals

In [187]:
dudes = [i for i in singlechip_ids if i[6] == 'M']
ladies = [i for i in singlechip_ids if i[6] == 'F']
eunuch = [i for i in singlechip_ids if i[6] == 'U']

In [165]:
singlechip.loc[dudes,].sum()

227234     280.0
58336     5835.0
777962     496.0
sum       6611.0
dtype: float64

In [167]:
singlechip.loc[ladies,].sum()

227234    10047.0
58336         2.0
777962      401.0
sum       10450.0
dtype: float64

In [169]:
singlechip.loc[eunuch,].sum()

227234     227.0
58336     1510.0
777962     833.0
sum       2570.0
dtype: float64

In [184]:
hdU = singlechip.loc[eunuch,][(chipperanimal['777962'] == 1)].index.tolist()

/home/jlh4df/.conda/envs/Imp3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [177]:
print(len(testf250snp50))
len([i for i in testf250snp50 if i[6] == 'M'])

1268


1197

In [188]:
print(ladies[:10])
#random.shuffle(ladies) #makes random order of the females it just once
#writef250_ladies = open('f250ladies_train.txt','w')
#for i in ladies:
#    writef250_ladies.write(i + '\n')
#writef250_ladies.close()
readf250_ladies = open('f250ladies_train.txt','r')
f250ladies = [i.strip() for i in readf250_ladies.readlines()]
f250ladies[:10]

['AANUSAF000017102723', 'AANUSAF000017102725', 'AANUSAF000017102733', 'AANUSAF000017102750', 'AANUSAF000017102754', 'AANUSAF000017102762', 'AANUSAF000017102763', 'AANUSAF000017102769', 'AANUSAF000017102772', 'AANUSAF000017152022']


['UMCUSAF000000103145',
 'UMCUSAF000000097566',
 'UMCUSAF000000109021',
 'UMCUSAF000000109625',
 'UMCUSAF000000098704',
 'UMCUSAF000000097935',
 'UMCUSAF000000097539',
 'AANUSAF000017667314',
 'UMCUSAF000000102904',
 'UMCUSAF000000100433']

In [191]:
reducedtest = hdU + f250ladies[:4000] + dudes
print(len(reducedtest))
print(len(singlechip))

11444
19631


In [193]:
writesmallsingle = open('Animals_Small_Single_100_11444.txt','w')
for i in reducedtest:
    writesmallsingle.write(' '.join([i,i,'\n']))
writesmallsingle.close()

# Now make a file with just these animals in it
 " plink --merge-list hwe_filtered/allfilesmerged2.txt --keep ./dataprepper/Animals_Small_Single_100_11444.txt --nonfounders  --cow --make-bed --out merged_files/single_chip_small_100"

In [ ]:
double = f250snp50.sample(600).index.tolist()
double[:10]

In [40]:
funcset = f250set.sample(400).index.tolist()

In [41]:
fiftyset = snp50set.sample(1000).index.tolist()
hdout = hdset.sample(1000).index.tolist()

# merge all assays togehter first

In [65]:
chipperanimal[(chipperanimal['sum'] > 1) &(chipperanimal['777962'] == 1) ].describe()

,227234,58336,777962,sum
count,233.000000,233.000000,233.0,233.000000
mean,0.695279,0.592275,1.0,2.287554
std,0.461280,0.492470,0.0,0.453596
min,0.000000,0.000000,1.0,2.000000
25%,0.000000,0.000000,1.0,2.000000
50%,1.000000,1.000000,1.0,2.000000
75%,1.000000,1.000000,1.0,3.000000
max,1.000000,1.000000,1.0,3.000000


In [66]:
threechips = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 1)]
threechips.head()
threechips.shape

(67, 4)

In [164]:
print(len(allanimals))
len(set(allanimals))
Counter(allanimals).most_common(10)

23114


[('AANUSAM000013531361', 3),
 ('AANUSAM000012227124', 3),
 ('AANUSAM000014257843', 3),
 ('AANUSAM000014000000', 3),
 ('AANUSAM000010776479', 3),
 ('AANUSAM000008781383', 3),
 ('AANUSAM000011216428', 3),
 ('AANUSAM000008477778', 3),
 ('AANUSAM000015150605', 3),
 ('AANUSAM000013627964', 3)]

In [273]:
len(idgrab(fname,set(fix)))

TypeError: idgrab() takes 1 positional argument but 2 were given

In [223]:
bobtest = idgrab('../raw_genotypes/777962.160906.75.imp_test.ID')

['AANUSAM000011393414',
 'AANUSAM000012886399',
 'AANUSAM000012588547',
 'AANUSAM000012363922',
 'AANUSAM000014000000',
 'AANUSAM000011462197',
 'AANUSAM000012502030',
 'AANUSAM000012240991',
 'AANUSAM000013300751',
 'AANUSAM000011994601']

In [245]:
bobanimal = chipperanimal.loc[bobtest]
bobanimal.sort_values(by='sum').head(6)

,227234,58336,777962,sum
AANUSAM000009911822,1.0,1.0,0.0,2.0
AANUSAM000012729973,0.0,1.0,1.0,2.0
AANUSAM000005221298,0.0,1.0,1.0,2.0
AANUSAM000011060761,1.0,1.0,0.0,2.0
AANUSAM000013085417,1.0,1.0,0.0,2.0
AANUSAM000015134182,1.0,1.0,0.0,2.0


In [253]:
! grep 'AANUSAM000005221298' ../../GEN_DUMP/*.100*.ID


../../GEN_DUMP/58336.161117.7744.100_A.ID: 1230 AANUSAM000005221298
../../GEN_DUMP/777962.161117.1681.100_A.ID: 80 AANUSAM000005221298


In [254]:
! grep 'AANUSAM000005221298' ../raw_genotypes/*.ID

../raw_genotypes/227234.160906.75.imp_test.ID: 62 AANUSAM000005221298
../raw_genotypes/58336.160906.75.imp_test.ID: 46 AANUSAM000005221298
../raw_genotypes/777962.160906.100.test_hd_A.ID: 84 AANUSAM000005221298
../raw_genotypes/777962.160906.75.imp_test.ID: 46 AANUSAM000005221298


In [249]:
for f in angframe.itertuples():

True

In [64]:
perChip.loc['227234']

count    12083
Name: 227234, dtype: int64

In [ ]:
for f in angframe.itertuples():
    print(f[1],f[-1],f[4])
    fname,touse,total,array = f[1],f[-1],f[4],f[2]
    fix = random.sample(range(total),int(touse))
    bix = idgrab(fname)
    perarray[array].append(bix)
    allanimals =allanimals + bix

In [3]:
from itertools import tee
def parwise(iterable):
	a,b = tee(iterable)
	next(b, None)
	return zip(a,b)

def chunks(end):
	return [str(i[0]+1)+ ' ' + str(i[1]) for i in parwise(range(0,end,5000000))]
	
rangedict = {'25': chunks(43879707)}

def chrchunker(WC):
	return rangedict[WC.chr][int(WC.chunk)]
	

In [6]:
rangedict['25'][3:10]

['15000001 20000000',
 '20000001 25000000',
 '25000001 30000000',
 '30000001 35000000',
 '35000001 40000000']

In [8]:
for i in parwise(range(0,10,3)):
    print(i)

(0, 3)
(3, 6)
(6, 9)


In [13]:
for i in range(0,10,3):
    print(i)

0
3
6
9


In [19]:
for i in tee(range(0,10,3)):
    print(i)

TypeError: 'itertools._tee' object is not subscriptable